# Travel time Tomography (TT) Forward Modeling - Simulating a survey and creating a velocity field file
## Run mode: 2    GPR example

This example build on the GPR example in Mesh Generation

The steps followed below are:   
1) Create the mesh  
2) Create fmm.inp and run the forward model. The file e4d.inp is not used to run a TT forward model.

The output options file creates a velocity file to be viewed in ViSit.

In [1]:
## 1) Create the mesh
# gpr.cfg is defined as the mesh configuation file in e4d.inp
import subprocess
import os
import sys

# create e4d.inp file to build the mesh
pre='bcd'

fileN='e4d.inp'
f1=open(fileN, 'w')
f1.write("1\n")    # run mode 1
f1.write(pre+".cfg\n") # mesh configuration file
f1.close()


cmd='mpirun -np 1 e4d' # assumes mpirun and e4d are in your path
result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
output=result.stdout.decode("utf-8") 

lines=output.split(sep='\n')
cont=False
for line in lines:
        #print (line)
        if 'Finished build' in line:
            print ('Mesh build okay')
            cont=True

   
## 2) Create fmm.inp and run the TT forward model 
if cont==True:
    fileN='fmm.inp'
    f2=open(fileN, 'w')
    f2.write("2\n")               # forward run mode 2
    f2.write(pre+".1.ele\n")      # mesh element file 
    f2.write(pre+".srv\n")        # survey file 
    f2.write("0.1\n")        # starting velocity file
    f2.write(pre+".out\n")        # output options
    f2.close()

    num_proc=20
    # mpirun -np num_proc_total e4d -fmm num_proc_fmm 
    # num_proc_total is the total number of processors
    # num_proc_fmm is the number of processors allocated for TT
    # For a TT forward run, there is no ERT inversion taking place so num_proc_total=num_proc_fmm
    
    cmd='mpirun -np '+str(num_proc)+' e4d -fmm '+str(num_proc) # for a TT run only, allocate the same number of processors to e4d and fmm
    
    result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)
    output=result.stdout.decode("utf-8") 

    lines=output.split(sep='\n')
    for line in lines:
            print (line)
            if 'WRITING SIMULATED TRAVEL-TIME FILE' in line:
                print ('Simulated file written!')
                


Mesh build okay
 Running FMM with:           20  processes

 ************************ WELCOME TO FMM ************************
 Copyright © 2014, Battelle Memorial Institute
 All rights reserved.
 Current date:  December 09, 2019
 Current time:  08:34:57
 Running on 0000020 processing cores
 Please refer to fmm.log for further logging information ...
 ****************************************************************
 There was a problem reading the number of zones to include
 in the forward travel time simulation after the mesh file name.
 Using all zones.
  FMM: EXECUTING FORWARD RUN
  WRITING TRAVEL TIME FILES
  WRITING SIMULATED SURVEY FILE: 0.1.srv
  Forward run complete.



In [2]:
# Create potential field visualization files
import subprocess

cmd='px -f bcd traveltime.100 tt100' # assumes px is in your path
result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)

output=result.stdout.decode("utf-8") 

lines=output.split(sep='\n')
for line in lines:
        print (line)


cmd='mv tt100.xmf visit/tt100.xmf' 
result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)

cmd='mv tt100.h5 visit/tt100.h5' 
result = subprocess.run(cmd, shell=True,stdout=subprocess.PIPE)



Creating new xmf and h5 file.....
Reading bcd.1.node
Reading bcd.1.ele
THE NUMBER OF NODES IS:56857
THE NUMBER OF ELEMENTS IS:343410
THE NUMBER OF ELEMENT ZONES IS:2
Creating tt100.h5 file
Recording potential file traveltime.100 at time stamp:0
Done writing tt100.h5 file
Finished build of tt100.xmf file



In [ ]:
# view the files in visit
import os
import subprocess

directory=os.getcwd()
if 'visit' not in directory:
    os.chdir('visit')

# get visit full path
fn='../../../../Visit_fullpathname'

f= open(fn,"r")
dir_Visit = f.readline().strip() 
subprocess.run([dir_Visit+'visit' , 'cli', '-s', 'script.py'])

os.chdir('../')
